# Criação, treino e teste do Modelo

## Instalando bibliotecas 

In [15]:
!pip install tensorflow

## Leitura e Verificação dos Dados

In [16]:
import pandas as pd

df = pd.read_parquet('DF_KNRS_COMUM_PROCESSADO.parquet')
print(df['TEM_FALHA_ROD'].value_counts())  # Verificando a distribuição da variável alvo

df = df.dropna()  # Removendo valores nulos
print(df.isnull().sum())  # Verificando novamente para garantir que não haja valores nulos

TEM_FALHA_ROD
0.0    64841
1.0     4653
Name: count, dtype: int64
KNR                       0
QTD_STATUS_1_OK           0
QTD_STATUS_1_NOK          0
QTD_STATUS_2_OK           0
QTD_STATUS_2_NOK          0
QTD_STATUS_718_OK         0
QTD_STATUS_718_NOK        0
TEMPO_MEDIO               0
MOTOR                     0
COR                       0
QTD_HALLE_                0
QTD_HALLE_AGUA            0
QTD_HALLE_BUY             0
QTD_HALLE_CAB             0
QTD_HALLE_DKA             0
QTD_HALLE_ESPC            0
QTD_HALLE_PROC            0
QTD_HALLE_PROF            0
QTD_HALLE_PVC             0
QTD_HALLE_ROD             0
QTD_HALLE_RUID            0
QTD_HALLE_TLUI            0
QTD_HALLE_ZP5             0
QTD_HALLE_ZP5A            0
QTD_HALLE_ZP6             0
QTD_HALLE_ZP61            0
QTD_HALLE_ZP62            0
QTD_HALLE_ZP7             0
QTD_HALLE_ZP8             0
QTD_HALLE_ZP82            0
QTD_HALLE_ZP8R            0
QTD_SGROUP_#MULTIVALUE    0
QTD_SGROUP_-2             0
QTD_SGROUP

## Pré-processamento dos dados

In [17]:
# Criando variáveis dummies
df = pd.get_dummies(df, columns=['COR', 'MOTOR'], drop_first=True)

# Removendo colunas desnecessárias
df = df.drop(columns=["KNR"])

## Balanceamento das Classes

In [18]:
from imblearn.under_sampling import RandomUnderSampler  # Importa o método RandomUnderSampler para balanceamento de classes

rus = RandomUnderSampler(random_state=0)  # Inicializa o RandomUnderSampler com uma semente de aleatoriedade fixa
X_resampled, y_resampled = rus.fit_resample(X, y)  # Aplica o balanceamento de classes aos dados, retornando as amostras balanceadas

## Dividindo o dataset entre treino e teste

In [19]:
from sklearn.model_selection import train_test_split  # Importa a função para dividir os dados em conjuntos de treino e teste

X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)  
# Divide os dados balanceados em conjuntos de treino (80%) e teste (20%), com uma semente de aleatoriedade fixa para reprodução

## Redimensionamento dos dados

In [20]:
X_train = X_train.values.reshape((X_train.shape[0], 1, X_train.shape[1]))  
# Altera a forma de X_train para (n amostras, 1, n características) para compatibilidade com redes neurais que esperam uma dimensão adicional

X_test = X_test.values.reshape((X_test.shape[0], 1, X_test.shape[1]))  
# Altera a forma de X_test de maneira semelhante

X_train = np.array(X_train, dtype=np.float32)  
# Converte X_train para um array NumPy com tipo de dado float32

y_train = np.array(y_train, dtype=np.float32)  
# Converte y_train para um array NumPy com tipo de dado float32

## Construção dos Modelos (LSTM e GRU)

### Modelo GRU

In [35]:
model_gru = Sequential()  # Cria um modelo sequencial, que é uma pilha linear de camadas

model_gru.add(GRU(50, input_shape=(X_train.shape[1], X_train.shape[2])))  
# Adiciona uma camada GRU com 50 unidades. A forma de entrada é (n timesteps, n características)

model_gru.add(Dropout(0.2))  
# Adiciona uma camada Dropout para prevenir overfitting, desativando aleatoriamente 20% dos neurônios durante o treinamento

model_gru.add(Dense(1, activation='sigmoid'))  
# Adiciona uma camada densa de saída com 1 neurônio e ativação sigmoide para uma tarefa de classificação binária

model_gru.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])  
# Compila o modelo usando o otimizador Adam com taxa de aprendizado de 0.001, a função de perda de entropia cruzada binária e a métrica de acurácia

/home/rizzi26/Documents/Github/faculdade/2024-2A-T08-EC07-G05/src/notebooks/venv/lib64/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [36]:
# Treinando o modelo GRU
history_gru = model_gru.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2)

Epoch 1/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.4918 - loss: 0.7550 - val_accuracy: 0.5165 - val_loss: 0.6958
Epoch 2/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5055 - loss: 0.7142 - val_accuracy: 0.5075 - val_loss: 0.7000
Epoch 3/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5204 - loss: 0.7045 - val_accuracy: 0.5165 - val_loss: 0.6937
Epoch 4/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5222 - loss: 0.6967 - val_accuracy: 0.5038 - val_loss: 0.6959
Epoch 5/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.4952 - loss: 0.7099 - val_accuracy: 0.4895 - val_loss: 0.6983
Epoch 6/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5073 - loss: 0.7004 - val_accuracy: 0.5150 - val_loss: 0.6945
Epoch 7/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5089 - loss: 0.6992 - val_accuracy: 0.5293 - val_loss: 0.6946
Epoch 8/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5332 - loss: 0.6910 - val_accu

### Modelo LSTM

In [22]:
# Definir o modelo LSTM
model_lstm = Sequential()  # Cria um modelo sequencial para o LSTM

# Camada LSTM
model_lstm.add(LSTM(50, input_shape=(X_train.shape[1], X_train.shape[2])))  
# Adiciona uma camada LSTM com 50 unidades. A forma de entrada é (n timesteps, n características)

model_lstm.add(Dropout(0.2))  
# Adiciona uma camada Dropout para prevenir overfitting, desativando aleatoriamente 20% dos neurônios durante o treinamento

# Camada de saída
model_lstm.add(Dense(1, activation='sigmoid'))  
# Adiciona uma camada densa de saída com 1 neurônio e ativação sigmoide para uma tarefa de classificação binária

# Compilar o modelo
model_lstm.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])  
# Compila o modelo usando o otimizador Adam com taxa de aprendizado de 0.001, a função de perda de entropia cruzada binária e a métrica de acurácia

# Resumo do modelo
model_lstm.summary()  # Exibe um resumo da arquitetura do modelo

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 50)             │        20,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 50)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 20,651 (80.67 KB)

 Trainable params: 20,651 (80.67 KB)

 Non-trainable params: 0 (0.00 B)

In [26]:
# Treinando o modelo LSTM
history_lstm = model_lstm.fit(X_train, y_train, epochs=200, batch_size=32, validation_split=0.2)

Epoch 1/200
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5734 - loss: 0.6768 - val_accuracy: 0.6291 - val_loss: 0.6633
Epoch 2/200
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6417 - loss: 0.6506 - val_accuracy: 0.8191 - val_loss: 0.6087
Epoch 3/200
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7311 - loss: 0.6087 - val_accuracy: 0.8829 - val_loss: 0.5740
Epoch 4/200
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7708 - loss: 0.5794 - val_accuracy: 0.9640 - val_loss: 0.5284
Epoch 5/200
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7775 - loss: 0.5541 - val_accuracy: 0.9279 - val_loss: 0.4762
Epoch 6/200
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8469 - loss: 0.4853 - val_accuracy: 0.9407 - val_loss: 0.4288
Epoch 7/200
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8489 - loss: 0.4424 - val_accuracy: 0.7860 - val_loss: 0.4905
Epoch 8/200
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7805 - loss: 0.4707 - val_accu

## Avaliando o Modelo

### GRU

In [37]:
# Avaliar o modelo GRU com os dados de teste
loss_gru, accuracy_gru = model_gru.evaluate(X_test, y_test)
print(f'Test Loss GRU: {loss_gru}')
print(f'Test Accuracy GRU: {accuracy_gru}')

# Fazer previsões com o modelo GRU
y_pred_gru = model_gru.predict(X_test)
y_pred_gru = (y_pred_gru > 0.5).astype(int)

# Exibir o relatório de classificação
print(classification_report(y_test, y_pred_gru))


53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 621us/step - accuracy: 0.9532 - loss: 0.0598
Test Loss GRU: 0.06438901275396347
Test Accuracy GRU: 0.9477477669715881
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
              precision    recall  f1-score   support

         0.0       0.91      1.00      0.95       841
         1.0       1.00      0.89      0.94       824

    accuracy                           0.95      1665
   macro avg       0.95      0.95      0.95      1665
weighted avg       0.95      0.95      0.95      1665



### LSTM

In [34]:
# Avaliar o modelo LSTM com os dados de teste
loss_lstm, accuracy_lstm = model_lstm.evaluate(X_test, y_test)
print(f'Test Loss LSTM: {loss_lstm}')
print(f'Test Accuracy LSTM: {accuracy_lstm}')

# Fazer previsões com o modelo LSTM
y_pred_lstm = model_lstm.predict(X_test)
y_pred_lstm = (y_pred_lstm > 0.5).astype(int)

# Exibir o relatório de classificação
print(classification_report(y_test, y_pred_lstm))

53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 505us/step - accuracy: 0.9537 - loss: 0.1107
Test Loss LSTM: 0.09792882204055786
Test Accuracy LSTM: 0.9615615606307983
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 576us/step
              precision    recall  f1-score   support

         0.0       0.93      1.00      0.96       841
         1.0       1.00      0.92      0.96       824

    accuracy                           0.96      1665
   macro avg       0.96      0.96      0.96      1665
weighted avg       0.96      0.96      0.96      1665

